In [1]:
import csv
import sys
import re
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')
review=pd.read_csv("/content/drive/MyDrive/nlp_data/review_topic_prob.csv")
business_data=pd.read_csv("/content/drive/MyDrive/nlp_data/yelp_academic_dataset_business.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df_business = business_data[business_data['categories'].notna()] #take out missing ones
#select restaurant data
restaurant_data=df_business[df_business["categories"].str.contains("Food",flags=re.IGNORECASE)|df_business["categories"].str.contains("Restaurant",flags=re.IGNORECASE)]
business_topic=restaurant_data[restaurant_data['state'] == 'PA']

In [4]:
business_topic.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,39.953949,-75.143226,4.0,245,1,"{'RestaurantsReservations': 'True', 'Restauran...","Sushi Bars, Restaurants, Japanese","{'Tuesday': '13:30-22:0', 'Wednesday': '13:30-..."
19,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147,39.943223,-75.162568,4.5,205,1,"{'NoiseLevel': ""u'quiet'"", 'GoodForMeal': ""{'d...","Korean, Restaurants","{'Monday': '11:30-20:30', 'Tuesday': '11:30-20..."
28,QdN72BWoyFypdGJhhI5r7g,Bar One,767 S 9th St,Philadelphia,PA,19147,39.939825,-75.157447,4.0,65,0,"{'Smoking': ""u'no'"", 'NoiseLevel': ""u'average'...","Cocktail Bars, Bars, Italian, Nightlife, Resta...","{'Monday': '16:0-0:0', 'Tuesday': '16:0-0:0', ..."


In [5]:
review.head()

,review_id,business_id,stars,text,processed_text,Topic,topic_name,sentiment_probability
0,KU_O5udG6zpxOg-VcAEodg,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,if you decide to eat here just be aware it is ...,decide eat aware go take hour begin end try mu...,3,waiting time,0.213844
1,AqPFMleE6RsU23_auESxiA,kxX2SOes4o-D3ZQBkiMRfA,5.0,wow yummy different delicious our favorite is ...,wow yummy different delicious favorite lamb cu...,2,Asian food,0.999990
2,JrIxlS1TzJ-iCu79ul40cQ,04UD14gamNjLY0IDYVhHJg,1.0,i am a long term frequent customer of this est...,long term frequent customer establishment go o...,3,waiting time,0.643294
3,_ZeMknuYdlQcUqng_Im3yg,LHSTtnW3YHCeUkRDGyJOyw,5.0,amazingly amazing wings and homemade bleu chee...,amazingly amaze wing homemade bleu cheese ribe...,1,dinner and dessert,1.000000
4,8JFGBuHMoiNDyfcxuWNtrA,RZtGWDLCAtuipwaZ-UfjmQ,4.0,good foodloved the gnocchi with marinara the b...,good foodloved gnocchi marinara bake eggplant ...,1,dinner and dessert,0.301108


In [5]:
null_count=review.isna().sum()
print(null_count)
null_count1=business_topic.isna().sum()
print(null_count1)

review_id                0
business_id              0
stars                    0
text                     0
processed_text           0
Topic                    0
topic_name               0
sentiment_probability    0
dtype: int64
business_id        0
name               0
address           96
city               0
state              0
postal_code        2
latitude           0
longitude          0
stars              0
review_count       0
is_open            0
attributes       163
categories         0
hours           2488
dtype: int64


In [6]:
topic_avg_probility = review.groupby(['business_id', 'topic_name'])['sentiment_probability'].mean().reset_index()
topic_avg_probility_pivot = topic_avg_probility.pivot_table(index='business_id', columns='topic_name', values='sentiment_probability')
topic_avg_probility_pivot.head()

topic_name,Asian food,Drinks and staff services,dinner and dessert,fastfood like burgers and sandwichs,pizza,waiting time
business_id,,,,,,
--ZVrH2X2QXBFdCilbirsw,NaN,0.985447,0.548777,0.975326,0.890246,0.305045
--epgcb7xHGuJ-4PUeSLAw,NaN,0.877059,NaN,0.660562,0.338319,0.139118
-0FX23yAacC4bbLaGPvyxw,NaN,0.878261,0.690408,0.503817,NaN,0.124575
-0M0b-XhtFagyLmsBtOe8w,NaN,0.860104,0.313896,NaN,NaN,0.007757
-0PN_KFPtbnLQZEeb23XiA,0.663055,0.817143,NaN,NaN,0.999993,0.052646


In [7]:
topic_avg_probility_pivot.shape

(15431, 6)

In [8]:
business_avg_probility = business_topic.merge(topic_avg_probility_pivot, on='business_id', how='left')

In [13]:
business_avg_probility.head()

,business_id,name,address,city,postal_code,latitude,longitude,stars,review_count,categories,hours,Asian food,Drinks and staff services,dinner and dessert,fastfood like burgers and sandwichs,pizza,waiting time
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,19107,39.955505,-75.155564,4.0,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",0.862782,0.881818,0.943388,0.839524,0.808303,0.194170
1,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,18054,40.338183,-75.471659,4.5,13,"Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",NaN,0.979084,NaN,NaN,NaN,0.007187
2,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,19106,39.953949,-75.143226,4.0,245,"Sushi Bars, Restaurants, Japanese","{'Tuesday': '13:30-22:0', 'Wednesday': '13:30-...",0.802486,0.893799,0.867186,0.909525,0.980291,0.135495
3,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,19147,39.943223,-75.162568,4.5,205,"Korean, Restaurants","{'Monday': '11:30-20:30', 'Tuesday': '11:30-20...",0.830053,0.964576,NaN,0.454482,0.881659,0.427742
4,QdN72BWoyFypdGJhhI5r7g,Bar One,767 S 9th St,Philadelphia,19147,39.939825,-75.157447,4.0,65,"Cocktail Bars, Bars, Italian, Nightlife, Resta...","{'Monday': '16:0-0:0', 'Tuesday': '16:0-0:0', ...",NaN,0.819815,0.841549,0.955814,0.541551,0.203192


In [14]:
business_avg_probility.shape

(15431, 17)

In [12]:
business_avg_probility=business_avg_probility.drop(['attributes','state','is_open'],axis=1)

In [15]:
null_count2=business_avg_probility.isna().sum()
print(null_count2)

business_id                               0
name                                      0
address                                  96
city                                      0
postal_code                               2
latitude                                  0
longitude                                 0
stars                                     0
review_count                              0
categories                                0
hours                                  2488
Asian food                             8286
Drinks and staff services              3945
dinner and dessert                     7688
fastfood like burgers and sandwichs    4821
pizza                                  2520
waiting time                           1440
dtype: int64


In [16]:
business_avg_probility.to_csv('/content/drive/MyDrive/nlp_data/business_topic.csv', index=False)